In [1]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
from IPython.display import display
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import os
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn import linear_model
import statsmodels.formula.api as smf
from sklearn.metrics import confusion_matrix
os.getcwd()
os.chdir('/Users/kylehoward/desktop/thinkful/thinkful-2') 
os.getcwd()

'/Users/kylehoward/Desktop/Thinkful/Thinkful-2'

In [2]:
data_path = ("FBI_data_4.csv")
fbi = pd.read_csv(data_path, usecols=[1,2,3,6,8,11,12])
fbi.columns = ['population', 'violent_crime', 'murder', 'robbery', 'prop_crime', 'gta', 'arsen']
fbi = fbi.replace([np.inf, -np.inf], np.nan).dropna()
fbi = fbi[1:]
#fbi = fbi.dropna()
fbi.head(186)

,population,violent_crime,murder,robbery,prop_crime,gta,arsen
1,2577,3,0,0,24,1,0.0
2,2846,3,0,0,16,0,0.0
6,1781,3,0,0,10,0,0.0
7,118296,107,1,31,2118,32,3.0
8,9519,9,0,4,210,6,1.0
9,18182,30,0,12,405,15,0.0
10,2053,0,0,0,39,1,0.0
11,4523,5,0,3,33,1,0.0
12,658,0,0,0,2,0,0.0
13,2522,2,0,0,5,1,0.0


In [3]:
fbi['population'] = fbi['population'].astype(int)
fbi['population_squared'] = fbi['population'] * fbi['population']
fbi['population_squared'].head(10)

1         6640929
2         8099716
6         3171961
7     13993943616
8        90611361
9       330585124
10        4214809
11       20457529
12         432964
13        6360484
Name: population_squared, dtype: int64

In [4]:

fbi['population_sqrt'] = fbi['population'] ** .5
fbi['population_sqrt'].head(10)

1      50.764161
2      53.347915
6      42.201896
7     343.941856
8      97.565363
9     134.840647
10     45.310043
11     67.253253
12     25.651511
13     50.219518
Name: population_sqrt, dtype: float64

In [5]:
fbi['murder'] = fbi['murder'].astype(int)
fbi['has_murder'] = np.where(fbi['murder']>=1, 1, 0)
fbi['has_murder'].head(10)

1     0
2     0
6     0
7     1
8     0
9     0
10    0
11    0
12    0
13    0
Name: has_murder, dtype: int64

In [6]:
fbi['robbery'] = fbi['robbery'].astype(int)
fbi['has_robbery'] = np.where(fbi['robbery']>=1, 1, 0)
fbi['has_robbery'].head(10)

1     0
2     0
6     0
7     1
8     1
9     1
10    0
11    1
12    0
13    0
Name: has_robbery, dtype: int64

In [7]:
regr = linear_model.LinearRegression()
fbi['prop_crime'] = fbi['prop_crime'].astype(int)
fbi['population_sqrt'] = fbi['population_sqrt'].astype(int)
fbi['violent_crime'] = fbi['violent_crime'].astype(int)
fbi['arsen'] = fbi['arsen'].astype(int)
fbi['gta'] = fbi['gta'].astype(int)
Y = fbi['prop_crime'].values.reshape(-1, 1)
X = fbi[['gta', 'population_sqrt', 'violent_crime', 'arsen']]
X_training = X[:100]
Y_training = Y[:100]
X_test = X[100:]
Y_test = Y[100:]
regr.fit(X_training, Y_training)
y_pred_test = regr.predict(X_test)
fbi['gta'].describe()

count    186.000000
mean      13.989247
std       57.506191
min        0.000000
25%        0.000000
50%        2.000000
75%        5.000000
max      609.000000
Name: gta, dtype: float64

In [8]:
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X_training, Y_training))


Coefficients: 
 [[ 17.30987145   4.02844667  -3.45162097  21.30677974]]

Intercept: 
 [-208.19966493]

R-squared:
0.781025167054


In [9]:
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X_test, Y_test))


Coefficients: 
 [[ 17.30987145   4.02844667  -3.45162097  21.30677974]]

Intercept: 
 [-208.19966493]

R-squared:
0.916290908524


In [10]:
print('Coefficients: \n', regr.coef_)
print(sklearn.metrics.mean_squared_error(Y_test, y_pred_test))
print(sklearn.metrics.r2_score(Y_test, y_pred_test))

Coefficients: 
 [[ 17.30987145   4.02844667  -3.45162097  21.30677974]]
148324.714285
0.916290908524


In [11]:
plt.scatter(fbi['gta'], Y,  color='black')
plt.xlim(0,1000)
plt.ylim(0,14000)
plt.loglog()
plt.show()

/Users/kylehoward/anaconda/envs/P3/lib/python3.6/site-packages/matplotlib/transforms.py:383: RuntimeWarning: invalid value encountered in double_scalars
  return points[1, 0] - points[0, 0]


OverflowError: cannot convert float infinity to integer